# Imports & Initializations

In [ ]:
from __future__ import division
import itertools, random, torch, os, time, pickle, psutil, gc, warnings
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable
import matplotlib.pyplot as plt
from datetime import datetime
import scipy.sparse as sp
import torch.nn as nn
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore")

path     = "/kaggle/input/iasdm2drl2022/"
path_out = "/kaggle/working/"

factors  = pd.read_csv(path+'factors_returns.csv')
strategy = pd.read_csv(path+'strategy_returns.csv')
device   = torch.device("cuda")

# LSTM Model 

In [ ]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, num_classes)
        self.hidden_state = None

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)

        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)

        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        self.hidden_state = h_out
        out = self.fc(h_out)

        return out

# Time Series Features

In [ ]:
# Convert df to numpy
def convert_df_to_np(df):
  vals = []
  for line in df.iterrows():
    vals.append((line[1].to_numpy()[:]))
  return np.nan_to_num(vals)

# Convert raw factors data to numpy array of shape (N,11)
def raw_to_np(df, val_init):
  vals = [val_init * np.ones(11)]
  for line in df.iterrows():
    vals.append((line[1].to_numpy()[1::]))
  return vals[1::]

# Normalize the factors
factors_processed = np.array(raw_to_np(factors,1))
factors_processed -= factors_processed.min()
factors_processed /= factors_processed.max()
factors_processed = factors_processed * 2 -1

factors_processed_df = pd.DataFrame(factors_processed)

# Compute factors features

factors_features = {}
def compute_features(factors_features, window_size):
  factors_features[window_size] = {}
  ### ROLLING STD
  vals   = factors_processed_df.rolling(window_size).std()
  factors_features[window_size]["rolling std"] = torch.from_numpy(convert_df_to_np(vals))

  ### ROLLING MEAN
  vals   = factors_processed_df.rolling(window_size).mean()
  factors_features[window_size]["rolling mean"] = torch.from_numpy(convert_df_to_np(vals))

  ### ROLLING MIN
  vals   = factors_processed_df.rolling(window_size).min()
  factors_features[window_size]["rolling min"] = torch.from_numpy(convert_df_to_np(vals))

  ### ROLLING MAX
  vals   = factors_processed_df.rolling(window_size).max()
  factors_features[window_size]["rolling max"] = torch.from_numpy(convert_df_to_np(vals))

compute_features(factors_features, 5)
compute_features(factors_features, 10)
compute_features(factors_features, 50)
compute_features(factors_features, 100)

# State Design

In [ ]:
# Create state given date and previous actions
def create_states(acs, date):

  #### Take all action_buffer_size previous actions
  acs = torch.from_numpy(np.array(acs))
  prev_acs = torch.empty(0)
  for i in range(1,action_buffer_size+1):
    if len(acs)>=i: prev_acs = torch.cat((prev_acs, acs[-i]), axis=0)
    else:           prev_acs = torch.cat((prev_acs, torch.ones(11)), axis=0)

  #### TIME SERIES ON ROLLING STD (feat_buffer_size * 11)
  feats = torch.empty(0)
  data  = factors_features[10]["rolling std"][:i]
  for i in range(1,feat_buffer_size+1):
    if len(data)>=i:
      feats = torch.cat((feats, data[-i]), axis=0)
    else:
      feats = torch.cat((feats, torch.zeros(11)), axis=0)

  #### GENERATE STATE
  state = torch.cat((feats, factors_features[10]["rolling mean"][i], factors_features[10]["rolling std"][i], factors_features[50]["rolling mean"][i], factors_features[50]["rolling std"][i],
                     torch.from_numpy(factors_processed[i].astype(np.float32)), prev_acs), axis=0).detach()

  return state

# DDPG Implementation

We used the following PyTorch implementation of DDPG: https://github.com/blackredscarf/pytorch-DDPG

In [ ]:
import numpy as np
import torch
import shutil
import torch.autograd as Variable

def soft_update(target, source, tau):
	"""
	Copies the parameters from source network (x) to target network (y) using the below update
	y = TAU*x + (1 - TAU)*y
	:param target: Target network (PyTorch)
	:param source: Source network (PyTorch)
	:return:
	"""
	for target_param, param in zip(target.parameters(), source.parameters()):
		target_param.data.copy_(
			target_param.data * (1.0 - tau) + param.data * tau
		)


def hard_update(target, source):
	"""
	Copies the parameters from source network to target network
	:param target: Target network (PyTorch)
	:param source: Source network (PyTorch)
	:return:
	"""
	for target_param, param in zip(target.parameters(), source.parameters()):
			target_param.data.copy_(param.data)


def save_training_checkpoint(state, is_best, episode_count):
	"""
	Saves the models, with all training parameters intact
	:param state:
	:param is_best:
	:param filename:
	:return:
	"""
	filename = str(episode_count) + 'checkpoint.path.rar'
	torch.save(state, filename)
	if is_best:
		shutil.copyfile(filename, 'model_best.pth.tar')


# Based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckActionNoise:

	def __init__(self, action_dim, mu = 0, theta = 0.15, sigma = 0.2):
		self.action_dim = action_dim
		self.mu = mu
		self.theta = theta
		self.sigma = sigma
		self.X = np.ones(self.action_dim) * self.mu

	def reset(self):
		self.X = np.ones(self.action_dim) * self.mu

	def sample(self):
		dx = self.theta * (self.mu - self.X)
		dx = dx + self.sigma * np.random.randn(len(self.X))
		self.X = self.X + dx
		return self.X


# use this to plot Ornstein Uhlenbeck random motion
if __name__ == '__main__':
	ou = OrnsteinUhlenbeckActionNoise(1)
	states = []
	for i in range(1000):
		states.append(ou.sample())
	import matplotlib.pyplot as plt


import numpy as np
import random
from collections import deque


class MemoryBuffer:

	def __init__(self, size):
		self.buffer = deque(maxlen=size)
		self.maxSize = size
		self.len = 0

	def sample(self, count):
		"""
		samples a random batch from the replay memory buffer
		:param count: batch size
		:return: batch (numpy array)
		"""
		batch = []
		count = min(count, self.len)
		batch = random.sample(self.buffer, count)

		s_arr = np.float32([arr[0] for arr in batch])
		a_arr = np.float32([arr[1] for arr in batch])
		r_arr = np.float32([arr[2] for arr in batch])
		s1_arr = np.float32([arr[3] for arr in batch])

		return s_arr, a_arr, r_arr, s1_arr

	def len(self):
		return self.len

	def add(self, s, a, r, s1):
		"""
		adds a particular transaction in the memory buffer
		:param s: current state
		:param a: action taken
		:param r: reward received
		:param s1: next state
		:return:
		"""
		transition = (s,a,r,s1)
		self.len += 1
		if self.len > self.maxSize:
			self.len = self.maxSize
		self.buffer.append(transition)

	def save(self):
		torch.save({"dict":self.buffer},path+"buffer.pth")

	def load(self):
		self.buffer = torch.load(path+"buffer.pth")["dict"]

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

EPS = 0.003

def fanin_init(size, fanin=None):
	fanin = fanin or size[0]
	v = 1. / np.sqrt(fanin)
	return torch.Tensor(size).uniform_(-v, v)

class Critic(nn.Module):

	def __init__(self, state_dim, action_dim):
		"""
		:param state_dim: Dimension of input state (int)
		:param action_dim: Dimension of input action (int)
		:return:
		"""
		super(Critic, self).__init__()

		self.lstm = LSTM(11, 11, 128, 1).to(device)

		self.state_dim = state_dim
		self.action_dim = action_dim

		self.fcs1 = nn.Linear(state_dim,256)
		self.fcs1.weight.data = fanin_init(self.fcs1.weight.data.size())
		self.fcs2 = nn.Linear(256,128)
		self.fcs2.weight.data = fanin_init(self.fcs2.weight.data.size())

		self.fca1 = nn.Linear(action_dim,128)
		self.fca1.weight.data = fanin_init(self.fca1.weight.data.size())

		self.fc2 = nn.Linear(256,128)
		self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())

		self.fc3 = nn.Linear(128,1)
		self.fc3.weight.data.uniform_(-EPS,EPS)

	def forward(self, state, action):
		"""
		returns Value function Q(s,a) obtained from critic network
		:param state: Input state (Torch Variable : [n,state_dim] )
		:param action: Input Action (Torch Variable : [n,action_dim] )
		:return: Value function : Q(S,a) (Torch Variable : [n,1] )
		"""



		state_1 = state[:,0:(feat_buffer_size*11)]



		state_2 = state[:,(feat_buffer_size*11):]



		out1    = self.lstm(state_1.reshape((-1,feat_buffer_size,11)))
		state   = torch.cat((out1,state_2),dim=1)

		s1 = F.relu(self.fcs1(state))
		s2 = F.relu(self.fcs2(s1))
		a1 = F.relu(self.fca1(action))
		x = torch.cat((s2,a1),dim=1)

		x = F.relu(self.fc2(x))
		x = self.fc3(x)

		return x


class Actor(nn.Module):

	def __init__(self, state_dim, action_dim, action_lim):
		"""
		:param state_dim: Dimension of input state (int)
		:param action_dim: Dimension of output action (int)
		:param action_lim: Used to limit action in [-action_lim,action_lim]
		:return:
		"""
		super(Actor, self).__init__()

		self.lstm = LSTM(11, 11, 128, 1).to(device)


		self.state_dim = state_dim
		self.action_dim = action_dim
		self.action_lim = action_lim.to(device)

		self.fc1 = nn.Linear(state_dim,256)
		self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())

		self.fc2 = nn.Linear(256,128)
		self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())

		self.fc3 = nn.Linear(128,64)
		self.fc3.weight.data = fanin_init(self.fc3.weight.data.size())

		self.fc4 = nn.Linear(64,action_dim)
		self.fc4.weight.data.uniform_(-EPS,EPS)

	def forward(self, state):
		"""
		returns policy function Pi(s) obtained from actor network
		this function is a gaussian prob distribution for all actions
		with mean lying in (-1,1) and sigma lying in (0,1)
		The sampled action can , then later be rescaled
		:param state: Input state (Torch Variable : [n,state_dim] )
		:return: Output action (Torch Variable: [n,action_dim] )
		"""

		state_1 = state[:,0:(feat_buffer_size*11)].to(device)
		state_2 = state[:,(feat_buffer_size*11):].to(device)
		out1    = self.lstm(state_1.reshape((-1,feat_buffer_size,11)))

		state   = torch.cat((out1,state_2),dim=1)

		x = F.relu(self.fc1(state))
		x = F.relu(self.fc2(x))
		x = F.relu(self.fc3(x))
		action = F.tanh(self.fc4(x))

		action = action * self.action_lim

		return action

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np
import math

BATCH_SIZE = 512
LEARNING_RATE = 0.00001
GAMMA = 0.99
TAU = 0.001

class Trainer:

  def __init__(self, state_dim, action_dim, action_lim, ram):
    """
    :param state_dim: Dimensions of state (int)
    :param action_dim: Dimension of action (int)
    :param action_lim: Used to limit action in [-action_lim,action_lim]
    :param ram: replay memory buffer object
    :return:
    """
    self.state_dim = state_dim
    self.action_dim = action_dim
    self.action_lim = action_lim
    self.ram = ram
    self.iter = 0
    self.noise = OrnsteinUhlenbeckActionNoise(self.action_dim)

    self.actor = Actor(self.state_dim, self.action_dim, self.action_lim).to(device)
    self.target_actor = Actor(self.state_dim, self.action_dim, self.action_lim).to(device)
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters(),LEARNING_RATE)

    self.critic = Critic(self.state_dim, self.action_dim).to(device)
    self.target_critic = Critic(self.state_dim, self.action_dim).to(device)
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters(),LEARNING_RATE)

    hard_update(self.target_actor, self.actor)
    hard_update(self.target_critic, self.critic)

  def get_exploitation_action(self, state):
    """
    gets the action from target actor added with exploration noise
    :param state: state (Numpy array)
    :return: sampled action (Numpy array)
    """
    state = Variable(torch.from_numpy(state))
    action = self.target_actor.forward(state.to(device)).detach()
    return action.data.cpu().numpy()

  def get_exploration_action(self, state):
    global nb_ep
    """
    gets the action from actor added with exploration noise
    :param state: state (Numpy array)
    :return: sampled action (Numpy array)
    """
    state = Variable(torch.from_numpy(state))
    action = self.actor.forward(state.to(device)).detach()

    factor   = random.randint(0,10)
    n_action = action.data.cpu().detach().numpy()
    n_action[0][factor] += (random.random()*2-1) * 0.01
    new_action = np.clip(n_action,-1,1)
    return new_action

  def optimize(self):
    """
    Samples a random batch from replay memory and performs optimization
    :return:
    """
    s1,a1,r1,s2 = self.ram.sample(BATCH_SIZE)

    s1 = Variable(torch.from_numpy(s1).to(device))
    a1 = Variable(torch.from_numpy(a1).to(device))
    r1 = Variable(torch.from_numpy(r1).to(device))
    s2 = Variable(torch.from_numpy(s2).to(device))

    # ---------------------- optimize critic ----------------------
    # Use target actor exploitation policy here for loss evaluation
    a2 = self.target_actor.forward(s2).detach()

    next_val = torch.squeeze(self.target_critic.forward(s2, a2).detach())

    # y_exp = r + gamma*Q'( s2, pi'(s2))
    y_expected = r1 + GAMMA*next_val
    # y_pred = Q( s1, a1)
    y_predicted = torch.squeeze(self.critic.forward(s1, a1))

    # compute critic loss, and update the critic
    loss_critic = F.smooth_l1_loss(y_predicted, y_expected)

    self.critic_optimizer.zero_grad()
    loss_critic.backward()
    self.critic_optimizer.step()

    # ---------------------- optimize actor ----------------------
    pred_a1 = self.actor.forward(s1)
    loss_actor = -1*torch.sum(self.critic.forward(s1, pred_a1))
    self.actor_optimizer.zero_grad()
    loss_actor.backward()
    self.actor_optimizer.step()

    soft_update(self.target_actor, self.actor, TAU)
    soft_update(self.target_critic, self.critic, TAU)

    # if self.iter % 100 == 0:
    # 	print 'Iteration :- ', self.iter, ' Loss_actor :- ', loss_actor.data.numpy(),\
    # 		' Loss_critic :- ', loss_critic.data.numpy()
    # self.iter += 1

  def save_models(self, episode_count):
    """
    saves the target actor and critic models
    :param episode_count: the count of episodes iterated
    :return:
    """
    torch.save(self.target_actor.state_dict(), path_out + str(episode_count) + '_actor.pt')
    torch.save(self.target_critic.state_dict(), path_out + str(episode_count) + '_critic.pt')
    print('Models saved successfully')

  def load_models(self, episode):
    """
    loads the target actor and critic models, and copies them onto actor and critic models
    :param episode: the count of episodes iterated (used to find the file name)
    :return:
    """
    self.actor.load_state_dict(torch.load(path_out + str(episode) + '_actor.pt'))
    self.critic.load_state_dict(torch.load(path_out + str(episode) + '_critic.pt'))
    hard_update(self.target_actor, self.actor)
    hard_update(self.target_critic, self.critic)
    print('Models loaded succesfully')

def reward(weights, factors_returns, strategy_returns):
    pred_returns = (1 + (weights * factors_returns).sum(axis=1)).cumprod(
        ).pct_change().fillna(0)
    tracking_error =  (pred_returns.values - strategy_returns.iloc[:,0].values
        ) * np.sqrt(250) * np.sqrt(weights.shape[1]+1)
    turn_over = 0.0020 * 365 * ((weights - weights.shift(1)).abs().fillna(0).values
        ) / ((weights.index[-1] -weights.index[0]).days) * np.sqrt(
        weights.shape[0] * (weights.shape[1]+1))
    error_terms = np.concatenate([tracking_error, turn_over.flatten()], axis=0)

    return -np.sqrt(np.mean(error_terms**2))


# Training

In [ ]:

# Import factors & strategy returns
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
factors_returns  = pd.read_csv(path+'factors_returns.csv', index_col=0,
                      parse_dates=True, date_parser=dateparse)
strategy_returns = pd.read_csv(path+'strategy_returns.csv', index_col=0,
                      parse_dates=True, date_parser=dateparse)

# Hyperparameters
use_gpu = True
action_buffer_size = 1
feat_buffer_size   = 25

MAX_EPISODES = 1000
MAX_STEPS    = 3058
MAX_BUFFER   = 100000

S_DIM = 11 + (11*4) + 11 + (11*1) # LSTM OUTPUT + FACTORS FEATURES + CURRENT FACTORS + PREVIOUS ACTIONS
A_DIM = 11
A_MAX = torch.ones(11)

ram     = MemoryBuffer(MAX_BUFFER)
trainer = Trainer(S_DIM, A_DIM, A_MAX, ram)

print(' State Dimensions :- ', S_DIM)
print(' Action Dimensions :- ', A_DIM)
print(' Action Max :- ', A_MAX)

ram     = MemoryBuffer(MAX_BUFFER)
trainer = Trainer(S_DIM, A_DIM, A_MAX, ram)

date = 0
acs  = []
training_rews = []

trainer.load_models(110)

for _ep in range(111,MAX_EPISODES):

  print('EPISODE :- ', _ep)
  date = 0
  acs  = []
  observation = create_states(acs, 0).detach()
  weights     = pd.DataFrame(index=[], columns =[], data=[])

  for r in range(MAX_STEPS):
    state = np.float32(observation)

    # Compute action (exploration or exploitation)
    if _ep%10 == 0: action = trainer.get_exploitation_action(state.reshape(1,-1))[0]
    else:           action = trainer.get_exploration_action(state.reshape(1,-1))[0]
    acs.append(action)

    # Update obs, next obs, done & date
    date += 1
    new_observation = create_states(acs, date).detach().numpy()
    observation     = new_observation
    done            = (date == MAX_STEPS)

    # Update weights
    new_row = pd.DataFrame(index=factors_returns[date-1:date].index, columns = factors_returns[date-1:date].columns, data=acs[date-1:date])
    weights = weights.append(new_row)

    # Compute immediate reward
    if date > 2:
      rew = reward(weights, factors_returns[:(date)], strategy_returns[:(date)])
    else:
      rew = 0

    if date % 100 == 0:
        print(rew)

    ############################
    if date == 3058:
        if _ep%10 != 0:
            print(f"Train - ",rew)
        else:
            print("#############")
            print(f"EVAL - ",rew)
            print("#############")
    ############################

    # Compute next state & update replay buffer (ram)
    new_state = np.float32(new_observation)
    ram.add(state, action, rew, new_state)

    if done:
      if _ep%10 == 0:
        training_rews.append(rew)
      break

  # check memory consumption and clear memory
  gc.collect()

  # Optimize when we have seen at least 1 episode
  if _ep > 1:
    for i in range(100):
      trainer.optimize()

  # Save model every 10 episodes
  if _ep != 0 and _ep%10 == 0:
    trainer.save_models(_ep)

# Submission

In [ ]:
MODEL_TO_SUBMIT = 160

In [ ]:
trainer = Trainer(S_DIM, A_DIM, A_MAX, ram)
trainer.load_models(MODEL_TO_SUBMIT)

date          = 0
acs           = []
training_rews = []
observation   = create_states(acs, 0).detach()
for r in range(3058):
  
  state  = np.float32(observation)
  action = trainer.get_exploitation_action(state.reshape(1,-1))[0]
  acs.append(action)

  date += 1

  if date != 3058:
    new_observation = create_states(acs, date).detach().numpy()
    observation     = new_observation
  if date == 3058:
    weights = pd.DataFrame(index=factors_returns[:(date)].index, columns = factors_returns[:(date)].columns, data=acs[-(date):])
    rew = reward(weights, factors_returns[:(date)], strategy_returns[:(date)])
    print(rew)

df = create_submission(weights,factors_returns)
df.to_csv(path_out+'final_submission.csv')